# Classical Random Circuits

Here we use classical circuits to get a handle on error rates.

This module that generates classical random circuits (that are not universal) on a graph which represents the QPU or QVM lattice. The basic idea is it will compute error rates of circuits as a function of depth and width.

The `width` of the circuit is the number of connected vertices on a particular subgraph.

The `depth` is defined in an unusual way. We consider a "depth 1" circuit to be a round of X gates randomly applied or not to a particular vertex AND a round of CNOTs randomly applied or not to each edge of the graph.

The circuit can also be executed in the X basis.

## Imports

In [ ]:
import random
import itertools
import networkx as nx
import numpy as np
import pandas as pd
import time
from scipy.spatial.distance import hamming
import scipy.interpolate

from matplotlib import pyplot as plt
from pyquil.api import get_qc, QuantumComputer
from pyquil.gates import CNOT, CCNOT, X, I, H, CZ, MEASURE, RESET
from pyquil.quilbase import Pragma

from forest_benchmarking.classical_random_circuits import *

## Get lattice

In [ ]:
# if you want to run on a "real lattice"
#from pyquil import *
#list_quantum_computers()
qc_perfect = get_qc("Aspen-1-15Q-A", as_qvm=True, noisy=False)
qc_noisy = get_qc("Aspen-1-15Q-A", as_qvm=True, noisy=True)
print(qc_perfect.name)
print(qc_noisy.name)

qc_perfect = get_qc("9q-square-qvm", as_qvm=True, noisy=False)
qc_noisy = get_qc("9q-square-qvm", as_qvm=True, noisy=True)

In [ ]:
nx.draw(qc_perfect.qubit_topology(),with_labels=True)

# Plot the distribution of sublattice widths

In [ ]:
G = qc_perfect.qubit_topology()
len(qc_perfect.qubit_topology())
# distribution of graph lengths
disty = []
for gdx in range(1,len(G.nodes)+1):
    listg = generate_connected_subgraphs(G,gdx)
    disty.append(len(listg))

cir_wid = list(range(1,len(G.nodes)+1))
plt.bar(cir_wid, disty, width=0.61, align='center')
plt.xticks(cir_wid)
plt.xlabel('sublattice / circuit width')
plt.ylabel('Frequency of Occurence')
plt.grid(axis='y', alpha=0.75)
plt.title('Distribution of sublattice widths')
disty

# Acquire data

In [ ]:
# with these parameters the cell below takes about 2 minutes
num_shots_per_circuit = 100
num_rand_subgraphs = 6
circuit_depth = 8
circuit_width = 6 #max = len(G.nodes)
in_x_basis = False
use_active_reset = False

In [ ]:
t0 = time.time()
data = get_random_classical_circuit_results(qc_perfect, qc_noisy, circuit_depth, circuit_width, num_rand_subgraphs, num_shots_per_circuit)
t1 = time.time()
total = t1-t0
print(total)

Now put the data into a dataframe

In [ ]:
df = pd.DataFrame(data)
df

# Plot a particular depth and width

In [ ]:
dep = 2
wid = 2

distz = get_hamming_dist(df, dep, wid)
averaged_distr = distz['hamming_dist'][0]
rand_ans_distr = hamming_dist_rand(wid,0)

In [ ]:
x_labels = np.arange(0, len(averaged_distr))
plt.bar(x_labels, averaged_distr, width=0.61, align='center')
plt.bar(x_labels, rand_ans_distr, width=0.31, align='center')
plt.xticks(x_labels)
plt.xlabel('Hamming Weight of Error')
plt.ylabel('Relative Frequency of Occurence')
plt.ylim([0,1])
plt.grid(axis='y', alpha=0.75)
plt.legend(['data','random'])
plt.title('Depth = {}, Width = {}'.format(dep,wid))
plt.show()

# For a particular width plot all depths

In [ ]:
wid = 4
hdis = get_hamming_dists_fn_depth(df, wid)

In [ ]:
for hdx in range(0, len(hdis)):
    averaged_distr = hdis.iloc[hdx]['hamming_dist']
    dep = hdis.iloc[hdx]['depth']
    rand_ans_distr = hamming_dist_rand(wid,0)
    x_labels = np.arange(0, len(averaged_distr))
    plt.subplot(1,len(hdis),hdx+1)
    plt.bar(x_labels, averaged_distr, width=0.61, align='center')
    plt.bar(x_labels, rand_ans_distr, width=0.31, align='center')
    plt.xticks(x_labels)
    plt.xlabel('Hamming Weight of Error')
    plt.ylabel('Relative Frequency of Occurence')
    plt.ylim([0,1])
    plt.grid(axis='y', alpha=0.75)
    plt.legend(['data','random'])
    plt.title('Depth = {}, Width = {}'.format(dep,wid))
plt.subplots_adjust(bottom=0.1, right=3.2, top=0.9)
plt.show()

Now we can study the sucess probablity, i.e. the zero hamming weight entry above as a function of depth. We first need to extract the data fron the data frame.

In [ ]:
num_bit_flips_allowed_from_answer = int(basement_function(np.log2(wid)-1))

pcheck = []
pcheck_rand = []
depth_vec = []
pcheck_log_errors = []
rand_pcheck_log_errors = []

for hdx in range(0, len(hdis)):
    averaged_distr = hdis.iloc[hdx]['hamming_dist']
    # probablity of getting the correct answer
    pcheck.append(averaged_distr[0])
    rand_ans_distr = hamming_dist_rand(wid,0)
    # probablity of getting the correct by randomly guessing
    pcheck_rand.append(rand_ans_distr[0])
    # error when you allow for a logarithmic number of bit flips from the true answer
    pcheck_log_errors.append(sum([averaged_distr[idx] for idx in range(0,num_bit_flips_allowed_from_answer+1)]))
    rand_pcheck_log_errors.append(sum([rand_ans_distr[idx] for idx in range(0,num_bit_flips_allowed_from_answer+1)]))
    dep = hdis.iloc[hdx]['depth']
    depth_vec.append(dep)

Next we will plot the sucess probablity of a circuit with a certain width as a function of depth. 

In [ ]:
plt.scatter(depth_vec,pcheck,label='Sucess Probablity')
plt.plot(depth_vec,pcheck_rand,label='random guess')
plt.ylim([0,1.05])
plt.xlabel('Depth')
plt.ylabel('Pr(success)')
plt.title('Pr(success) vs Depth for Width = {}'.format(wid))
plt.legend()
plt.show()

Now we allow a logarithmic number of bits to flip from the correct answer and call all such instances "success". E.g.

The logarithmic number of bits that we allow to flip is defined by the "basement" of 

$\log_2 ({\rm number\ of\ bits}) -1$

where the basement of a number is ${\rm basement(number)} = 0$ if number$<=0$ and ${\rm basement(number)} = {\rm floor (number)}$.


Supose we have a circuit of width 4, this means correct string has four bits, e.g. 1010. Then a logarithmic number of flips is $\log_2(4)-1 = 1$.

So any string with hamming weight zero or one counts as a success.

Such error metrics might be important in noisy near term algorithms where getting the exact answer is not vital.

In [ ]:
plt.figure()
plt.scatter(depth_vec,pcheck_log_errors,label='Sucess Probablity + log errors')
plt.plot(depth_vec,rand_pcheck_log_errors,label='random guess + log errors')
plt.ylim([0,1.05])
plt.xlabel('Depth')
plt.ylabel('Pr(success+log errors)')
plt.title('Pr(success+log errors) vs Depth for Width = {}'.format(wid))
plt.legend()
plt.show()

# Plot depth = width

In [ ]:
max_idx = min([max(df['depth']),max(df['width'])])

for idx in range(1,max_idx+1):
    hdis = get_hamming_dist(df, idx, idx)
    averaged_distr = hdis['hamming_dist'][0]
    dep = hdis['depth'][0]
    wid = hdis['width'][0]
    rand_ans_distr = hamming_dist_rand(idx,0)
    x_labels = np.arange(0, len(averaged_distr))
    plt.subplot(1,max_idx,idx)
    plt.bar(x_labels, averaged_distr, width=0.61, align='center')
    plt.bar(x_labels, rand_ans_distr, width=0.31, align='center')
    plt.xticks(x_labels)
    plt.xlabel('Hamming Weight of Error')
    plt.ylabel('Relative Frequency of Occurence')
    plt.ylim([0,1])
    plt.grid(axis='y', alpha=0.75)
    plt.legend(['data','random'])
    plt.title('Depth = {}, Width = {}'.format(dep,wid))
plt.subplots_adjust(bottom=0.1, right=3.2, top=0.9)
plt.show()

# Plot success probablity landscape

This is just the success probablity as a function of depth and width.

In [ ]:
points = np.vstack((df['depth'].values, df['width'].values)).T
points.shape

In [ ]:
# df['hamming_dist'][0] returns the array
# df['hamming_dist'][0][0] returns the first element of the array
values = np.asarray([df['hamming_dist'][idx][0] for idx in df.index])
values

In [ ]:
rand_width = [hamming_dist_rand(idx, 0)[0] for idx in range(1,circuit_width+1)]
rand_width

values_rand = np.asarray([item for sublist in [rand_width for ddx in range(1,circuit_depth+1)] for item in sublist])
values_rand.shape

In [ ]:
xx, yy, zz = interpolate_2d_landscape(points, values)

ax = plt.gca()
img = ax.imshow(zz, interpolation='none',
                extent=(xx[0, 0], xx[0, -1], yy[0, 0], yy[-1, 0]),
                cmap='viridis', origin='lowerleft', norm=plt.Normalize(None, None))
plt.colorbar(img, ax=ax)
plt.xlabel('Depth')
plt.ylabel('Width')
plt.title('Success Probability')
plt.show()

In [ ]:
print(points.shape)
xx, yy, zz = interpolate_2d_landscape(points, values_rand)

ax = plt.gca()
img = ax.imshow(zz, interpolation='none',
                extent=(xx[0, 0], xx[0, -1], yy[0, 0], yy[-1, 0]),
                cmap='viridis', origin='lowerleft', norm=plt.Normalize(None, None))
plt.colorbar(img, ax=ax)
plt.xlabel('Depth')
plt.ylabel('Width')
plt.title('Success Probability for Random guess')
plt.show()